In [1]:
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import math
import csv


In [2]:
PARENT_FOLDER = "/Users/nadia/Work/College_Courses/ImmersiveScholar2023/glucose-prediction/Dataset/CSV"
SAVE_FOLDER = "/Users/nadia/Work/College_Courses/ImmersiveScholar2023/glucose-prediction/RP_Images/"
NPY_FOLDER = f"/Users/nadia/Work/College/ImmersiveScholar2023/GlucosePredictionProject/Dataset/NUMPY"


In [13]:

def read_combo_data(file_path, num_samples, num_readings_per_sample):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  
        for row in csv_reader:
            if len(row) == 4 and all(cell.strip() for cell in row):
                x, y, z = float(row[1]), float(row[2]), float(row[3])
                data.append([x, y, z])

    total_readings = len(data)
    if total_readings < num_samples * num_readings_per_sample:
        print("Not enough data points to create the desired shape.")
        return None

    combo_array = np.array(data[:num_samples * num_readings_per_sample])
    combo_array = combo_array.reshape(num_samples, num_readings_per_sample, 3)
    return combo_array


for i in range(1, 15): 
    PARTICIPANT_FOLDER = f"P{i}"
    COMBO_FILE = f"COMBO_{str(i).zfill(3)}.csv"
    CSV_FILE = os.path.join(PARENT_FOLDER, PARTICIPANT_FOLDER, COMBO_FILE)
    NPY_FILE = f"/Users/nadia/Work/College_Courses/ImmersiveScholar2023/glucose-prediction/Dataset/NUMPY/P{i}/COMBO_{str(i).zfill(3)}.npy"
    data = pd.read_csv(CSV_FILE)
    print(len(data))
    num_samples = len(data) // 1200
    print(num_samples)
    num_readings_per_sample = 1200
    combo_data = read_combo_data(CSV_FILE, num_samples, num_readings_per_sample)
    if combo_data is not None:
        print("Shape of combo data:", combo_data.shape)
    np.save(NPY_FILE, combo_data)
    print(combo_data)   


158565
132
Shape of combo data: (132, 1200, 3)
[[[ 3.0210000e+01 -3.7500000e-03  0.0000000e+00]
  [ 3.0210000e+01  2.1650000e+00  1.2810000e-03]
  [ 3.0210000e+01  1.2955000e+01  2.6905000e-02]
  ...
  [ 3.1010000e+01 -2.5931250e+00  8.5839000e-02]
  [ 3.1010000e+01 -5.6937500e-01  8.7120000e-02]
  [ 3.1010000e+01  1.0746250e+01  8.5839000e-02]]

 [[ 3.1010000e+01  4.5706250e+00  8.7120000e-02]
  [ 3.1010000e+01 -6.0243750e+00  8.7120000e-02]
  [ 3.1010000e+01  6.0687500e+00  8.4558000e-02]
  ...
  [ 3.0870000e+01  5.0731250e+00  6.4699400e-01]
  [ 3.0870000e+01  1.3433125e+01  6.4699400e-01]
  [ 3.0870000e+01 -6.9812500e-01  6.4187000e-01]]

 [[ 3.0850000e+01  1.7560000e+01  6.4187000e-01]
  [ 3.0850000e+01 -1.6181250e+00  6.3674500e-01]
  [ 3.0850000e+01 -3.2137500e+00  6.3546400e-01]
  ...
  [ 3.0670000e+01  4.3965625e+01  1.3196100e-01]
  [ 3.0670000e+01  4.2118750e+00  1.3068000e-01]
  [ 3.0670000e+01 -2.0357500e+01  1.3196100e-01]]

 ...

 [[ 3.6590000e+01 -3.0868750e+00  2.80845

In [21]:
def phase_diff(a, b):

    fft_a = np.fft.fft(a)
    fft_b = np.fft.fft(b)
   
    phase_a = np.angle(fft_a)
    phase_b = np.angle(fft_b)

    phase_diff = phase_b - phase_a

    return phase_diff

def angular_diff(a, b):  
 
    pd = phase_diff(a, b)
    dot_product = np.dot(pd, [1, 1])
    magnitude_pd = np.sqrt(np.dot(pd, pd))
    magnitude_v = np.sqrt(2) 
    cos_angle = dot_product / (magnitude_pd * magnitude_v)
    angle = np.arccos(cos_angle)

    return angle

def varRP_freq(data, dim):
    x = []
    if dim == 'x':
        for j in range(150): 
            x.append(data[j][0])
    elif dim == 'y':
        for j in range(150): 
            x.append(data[j][1])
    elif dim == 'z':
        for j in range(150): 
            x.append(data[j][2])

    
    s = []
  
    for i in range(len(x) - 2):
        pd = phase_diff(x[i:i + 2], x[i + 1:i + 3]) 
        s.append(pd)

    dimR = len(s)
   
    R = np.eye(dimR)
    for i in range(dimR):
        for j in range(dimR):
            ang_diff = angular_diff(s[i], s[j])
            if ang_diff >3* np.pi/4:  
                sign = -1.0
            else:
                sign = 1.0
            R[i][j] = sign * np.mean(np.abs(phase_diff(s[i], s[j])))

    return R


def RemoveZero(l):
    nonZeroL = [] 
    
    for i in range(len(l)):
        if l[i] != 0.0: 
            nonZeroL.append(l[i]) 
    return nonZeroL

def NormalizeMatrix(_r):
    dimR = _r.shape[0]
    h_max = []
    for i in range(dimR):
        h_max.append(max(_r[i]))
    _max =  max(h_max)
    h_min = []
    for i in range(dimR):
        non_zero_elements = RemoveZero(_r[i])
        if len(non_zero_elements) > 0:
            h_min.append(min(non_zero_elements))
    _min =  min(h_min) if h_min else 0  # NEW! If h_min is empty, set _min to 0 or any default value
    _max_min = _max - _min
    _normalizedRP = np.zeros((dimR,dimR))
    for i in range(dimR):
        for j in range(dimR):
            _normalizedRP[i][j] = _r[i][j]/_max_min
    return _normalizedRP


def RGBfromRPMatrix_of_XYZ(X,Y,Z):
    if X.shape != Y.shape or X.shape != Z.shape or Y.shape != Z.shape:
        print('XYZ should be in same shape!') 
        return 0
    
    dimImage = X.shape[0] 
    newImage = np.zeros((dimImage,dimImage,3)) 

    for i in range(dimImage):
        for j in range(dimImage):
            _pixel = [] 
            _pixel.append(X[i][j]) 
            _pixel.append(Y[i][j])
            _pixel.append(Z[i][j]) 
            newImage[i][j] = _pixel 
    return newImage


def SavevarRP_XYZ(x, title,i, normalized):
    _r = varRP_freq(x, 'x') 
    _g = varRP_freq(x, 'y') 
    _b = varRP_freq(x, 'z') 

    plt.close('all') 
    plt.axis('off') 
    plt.margins(0, 0) 
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator()) 

    if normalized:
        newImage = RGBfromRPMatrix_of_XYZ(NormalizeMatrix(_r), NormalizeMatrix(_g), NormalizeMatrix(_b))
        plt.imshow(newImage) 
        plt.savefig(os.path.join(SAVE_FOLDER,f'P{i}',f'{title}_{j}.png'), bbox_inches='tight', pad_inches=0)
        plt.close('all') 
    else:
        newImage = RGBfromRPMatrix_of_XYZ(_r, _g, _b)
        plt.imshow(newImage)
        plt.savefig(os.path.join(SAVE_FOLDER,f'P{i}',f'{title}_{j}.png'), bbox_inches='tight', pad_inches=0)
        plt.close('all')


In [22]:

for i in range(5, 15): 
    NPY_FILE = f"/Users/nadia/Work/College_Courses/ImmersiveScholar2023/glucose-prediction/Dataset/NUMPY/P{i}/COMBO_{str(i).zfill(3)}.npy" 
    combo_data = np.load(NPY_FILE)
    title = f'COMBO_{str(i).zfill(3)}'
    for j in range(len(combo_data)):
            SavevarRP_XYZ(combo_data[j], title, i, normalized=1)
          




/var/folders/zg/rdp2_g2s3nl0_bqzgg3499z40000gn/T/ipykernel_78916/1501071234.py:19: RuntimeWarning: invalid value encountered in scalar divide
  cos_angle = dot_product / (magnitude_pd * magnitude_v)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.5..2.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.5..2.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.5..2.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.5..2.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.5..2.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.5..2.0].
Cli